### Task 1. 

#### MapReduce

Use MapReduce to calculate the average housing price in each zip code and filter out records in which gross sqft is less than 500 and sale price is less than 100000. Round the average housing price to 2 decimals, and sort the output by housing price ascending.

In [1]:
!wget -O housingSalseSample.csv https://github.com/CUSP2021ADS/Data/raw/main/housingSalseSample.csv

--2021-03-07 21:50:25--  https://github.com/CUSP2021ADS/Data/raw/main/housingSalseSample.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CUSP2021ADS/Data/main/housingSalseSample.csv [following]
--2021-03-07 21:50:25--  https://raw.githubusercontent.com/CUSP2021ADS/Data/main/housingSalseSample.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6167298 (5.9M) [text/plain]
Saving to: ‘housingSalseSample.csv’

housingSalseSample. 100%[===================>]   5.88M  7.56MB/s    in 0.8s    

2021-03-07 21:50:26 (7.56 MB/s) - ‘housingSalseSample.csv’ saved [6167298/6167298]



In [2]:
from functools import reduce
import csv

def returnInfo(record):
    return(record['ZIP CODE'],record['GROSS SQUARE FEET'],record['SALE PRICE'])

def sum_sqft_price(result,record):
    zipcode, sqft, price = record[0], record[1], record[2]
    if zipcode not in result:
        result[zipcode] = (float(sqft),float(price))
    else:
        result[zipcode] = (result[zipcode][0]+float(sqft),result[zipcode][1]+float(price))
    return result

In [3]:
with open('housingSalseSample.csv', 'r') as fi:
    reader = csv.DictReader(fi)

    output = sorted(map(lambda x: [int(float(x[0])),round(x[1],2)],
                 map(lambda x: (x[0],x[1][1]/x[1][0]),
                 list(reduce(sum_sqft_price,
                             filter(lambda x: float(x[1])>500 and float(x[2])>100000,
                                    filter(lambda x: len(x[0])*len(x[1])*len(x[2])>0, 
                                           list(map(returnInfo,reader)))),{}).items()))), key = lambda x: x[1])

In [4]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['zipcode','price']).to_csv('output1.csv',index=False)

grading(output)

### Task 2. 
#### multiprocessing and Chunk

In [5]:
!wget -O ADSSession6Task2Data.csv https://raw.githubusercontent.com/CUSP2021ADS/Data/main/ADSSession6Task2Data.csv

--2021-03-07 21:50:27--  https://raw.githubusercontent.com/CUSP2021ADS/Data/main/ADSSession6Task2Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279369 (273K) [text/plain]
Saving to: ‘ADSSession6Task2Data.csv’

ADSSession6Task2Dat 100%[===================>] 272.82K  --.-KB/s    in 0.1s    

2021-03-07 21:50:27 (2.34 MB/s) - ‘ADSSession6Task2Data.csv’ saved [279369/279369]



Apply the chunk and MapReduce method to sum up each type's value in the synthetic dataset—only keep records which $Select$ value is 1, and read 1000 lines in each chunk. Use multiprocessing package to deploy the task to multi-cores. The output should only have two columns: type and total value.

Hint: filter function is not available in multiprocessing, implement an if-else function in the map or reduce step.

In [6]:
from multiprocessing import Pool

def returnValue(record):
    return (record['Type'],record['Value'],record['Select'])

    
def sumValue(result,record): #add rides from a new record to the dictionary accumilating type-origin-destination-time ridership
    if str(record[2]) == '1':
        result[record[0]] = result.get(record[0],0) + float(record[1])

    return result

In [7]:
with open('ADSSession6Task2Data.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    chunk = []
    result = []
    # read 100000 rows in each chunk
    chunksize = 1000
    pool = Pool(8)
    for i, line in enumerate(reader):
        if (i % chunksize == 0 and i > 0):
            chunkOutput = reduce(sumValue,
                        pool.map(returnValue,chunk),
                        {}).items()
            result += chunkOutput
            chunk = []
            
        chunk.append(line)
        
    chunkOutput = reduce(sumValue,
            pool.map(returnValue,chunk),
            {}).items()
    result += chunkOutput
    pool.close()

In [8]:
def lastReducer(finalRecord,record):
    finalRecord[record[0]] = finalRecord.get(record[0],0) + float(record[1])
    return finalRecord

output = list(reduce(lastReducer,result,{}).items())
output

[('3', 294.32549925839936),
 ('0', 306.7099300824752),
 ('1', 316.7932728146583),
 ('9', 291.68146927692595),
 ('6', 302.75696574193273),
 ('7', 280.6029945361827),
 ('8', 274.5970828385255),
 ('5', 302.4264220421208),
 ('4', 320.753814015207),
 ('2', 286.0854868555854)]

In [9]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['Type','Value']).to_csv('output2.csv',index=False)

grading(output)